# Tensorflow Backdoor Poisoning MNIST Demo 

This demo will cover a basic backdoor poisoning attack on an MNIST-LeNet model.
The following two sections cover experiment setup and is similar across all demos.
To access demo results in MlFlow, please follow the general experiment setup steps outlined in `basic-mlflow-demo`.

## Setup: Experiment Name and MNIST Dataset

Here we will import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

**Important: Users will need to verify or update the following parameters:**

- Ensure that the `USERNAME` parameter is set to your own name.
- Ensure that the `DATASET_DIR` parameter is set to the location of the MNIST dataset directory. Currently set to `/nfs/data/Mnist` as the default location.
- (Optional) Set the `EXPERIMENT_NAME` parameter to your own preferred experiment name.

Other parameters can be modified to alter the RESTful API and MLFlow tracking addresses. 

In [ ]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Filter out warning messages
warnings.filterwarnings("ignore")

# Please enter custom username here.
USERNAME = "howard"

# Ensure that the dataset location is properly set here.
DATASET_DIR = "/nfs/data/Mnist"

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_mnist_backdoor_poison"

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Dioptra API ports
RESTAPI_PORT = "30080"
MLFLOW_TRACKING_PORT = "35000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the DIOPTRA_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Path to custom task plugins archives
CUSTOM_PLUGINS_POISONING_TAR_GZ = Path("custom-plugins-poisoning.tar.gz")

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `MLproject` file.
To run these entrypoints within Dioptra's architecture, we need to package those files up into an archive and submit it to the Dioptra RESTful API to create a new job.
For convenience, the `Makefile` provides a rule for creating the archive file for this example, just run `make workflows`,

In [ ]:
%%bash

# Create the workflows.tar.gz file
make workflows

To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the Dioptra RESTful API using the HTTP protocol.
We connect using the client below, which uses the environment variable `DIOPTRA_RESTAPI_URI` to figure out how to connect to the Dioptra RESTful API,

In [ ]:
restapi_client = utils.DioptraClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

We should also check which queues are available for running our jobs to make sure that the resources that we need are available.
The code below queries the Dioptra API and returns a list of active queues.

In [ ]:
restapi_client.list_queues()

This example also makes use of the `custom_poisoning_plugins` package stored locally under the `task-plugins/dioptra_custom/custom_poisoning_plugins` directory.
To register these custom task plugins, we first need to package them up into an archive.
For convenience, the `Makefile` provides a rule for creating the custom task plugins archive file, just run `make custom-plugins`,

In [ ]:
%%bash

# Create the workflows.tar.gz file
make custom-plugins

Now that the custom task plugin package is packaged into an archive file, next we register it by uploading the file to the REST API.
Note that we need to provide the name to use for custom task plugin package, this name must be unique under the custom task plugins namespace.
For a full list of the custom task plugins, use `restapi_client.restapi_client.list_custom_task_plugins()`.

In [ ]:
restapi_client.delete_custom_task_plugin(name="custom_poisoning_plugins")
response_custom_plugins = restapi_client.get_custom_task_plugin(name="custom_poisoning_plugins")

if response_custom_plugins is None or "Not Found" in response_custom_plugins.get("message", []):
    response_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="custom_poisoning_plugins",
        custom_plugin_file=CUSTOM_PLUGINS_POISONING_TAR_GZ,
    )

response_custom_plugins

If at any point you need to update one or more files within the `custom_poisoning_plugins` package, you will need to unregister/delete the custom task plugin first using the REST API.
This can be done as follows,

```python
# Delete the 'custom_poisoning_plugins' custom task plugin package
restapi_client.delete_custom_task_plugin(name="custom_poisoning_plugins")
```

The following helper functions will recheck the job responses until the job is completed or a run ID is available. 
The run ID is needed to link dependencies between jobs.

In [ ]:
def mlflow_run_id_is_not_known(job_response):
    return job_response["mlflowRunId"] is None and job_response["status"] not in [
        "failed",
        "finished",
    ]


def get_run_id(job_response):
    while mlflow_run_id_is_not_known(job_response):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
        
    return job_response


def wait_until_finished(job_response):
    # First make sure job has started.
    job_response = get_run_id(job_response)
    
    # Next re-check job until it has stopped running.
    while (job_response["status"] not in ["failed", "finished"]):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
    
    return job_response


# Helper function for viewing MLflow results.
def get_mlflow_results(job_response):
    mlflow_client = MlflowClient()
    job_response = wait_until_finished(job_response)
    
    if(job_response['status']=="failed"):
        return {}
    
    run = mlflow_client.get_run(job_response["mlflowRunId"])  
    
    while(len(run.data.metrics) == 0):
        time.sleep(1)
        run = mlflow_client.get_run(job_response["mlflowRunId"])
        
    return run


def print_mlflow_results(response):
    results = get_mlflow_results(response)
    pprint.pprint(results.data.metrics)

## MNIST Training: Baseline Model

Next, we need to train our baseline model that will serve as a reference point for the effectiveness of our attacks.
We will be submitting our job to the `"tensorflow_gpu"` queue.

In [ ]:
response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        f"-P register_model_name={EXPERIMENT_NAME}_le_net",
        "-P model_architecture=le_net",
        "-P epochs=30",
        f"-P data_dir_training={DATASET_DIR}/training",
        f"-P data_dir_testing={DATASET_DIR}/testing",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",    
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)

response_le_net_train = get_run_id(response_le_net_train)
print_mlflow_results(response_le_net_train)

### Generating Poisoned Images

Now we will create our set of poisoned images.
Start by submitting the poison generation job below.

In [ ]:
%%bash

# Create the workflows.tar.gz file
make workflows

In [ ]:
## Create poisoned test images.
response_gen_poison_le_net_test = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_poison_data",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={DATASET_DIR}/testing",
            "-P batch_size=100",
            "-P target_class=1",
            "-P poison_fraction=1",
            "-P label_type=test"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)

print("Backdoor poison attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_gen_poison_le_net_test)
print("")

response_gen_poison_le_net_test = get_run_id(response_gen_poison_le_net_test)

In [ ]:
## Create poisoned training images.
response_gen_poison_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_poison_data",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={DATASET_DIR}/training",
            "-P batch_size=100",
            "-P target_class=1",
            "-P poison_fraction=0.1",
            "-P label_type=train"
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)

print("Backdoor poison attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_gen_poison_le_net_train)
print("")

response_gen_poison_le_net_train = get_run_id(response_gen_poison_le_net_train)

In [ ]:
# Train new model on poisoned training images
response_le_net_poison_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        f"-P register_model_name={EXPERIMENT_NAME}_data_poison_le_net",
        "-P model_architecture=le_net",
        "-P epochs=30",
        f"-P data_dir_training={DATASET_DIR}/training",
        f"-P data_dir_testing={DATASET_DIR}/testing",
        "-P load_dataset_from_mlruns=true",
        f"-P dataset_run_id_training={response_gen_poison_le_net_train['mlflowRunId']}",
        "-P adv_tar_name=adversarial_poison.tar.gz",
        "-P adv_data_dir=adv_poison_data"
    ]),
    queue="tensorflow_gpu",
    depends_on=response_gen_poison_le_net_train ["jobId"],
    timeout="1h",
    
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_poison_model)

response_le_net_poison_model = get_run_id(response_le_net_poison_model)    

In [ ]:
# Inference: Regular model on poisoned test images.
response_infer_reg_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [            
            f"-P run_id={response_gen_poison_le_net_test['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            "-P adv_tar_name=adversarial_poison.tar.gz",
            "-P adv_data_dir=adv_poison_data",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_gen_poison_le_net_train["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_reg_model)

print_mlflow_results(response_infer_reg_model)

In [ ]:
# Inference: Poisoned model on poisoned test images.
response_infer_pos_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [            
            f"-P run_id={response_gen_poison_le_net_test['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_data_poison_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            "-P adv_tar_name=adversarial_poison.tar.gz",
            "-P adv_data_dir=adv_poison_data",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_poison_model["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_pos_model)

print_mlflow_results(response_infer_pos_model)

## Defending against the adversarial backdoor poisoning attack

Now we will explore available defenses on the adversarial backdoor poisoning attack.
The following three jobs will run a selected defense (spatial smoothing, gaussian augmentation, or jpeg compression) and evaluate the defense on the baseline and backdoor trained models.

- The first job uses the selected defense entrypoint to apply a preprocessing defense over the poisoned test images.
- The second job runs the defended images against the poisoned backdoor model.
- The final job runs the defended images against the baseline model.

Ideally the defense will not impact the baseline model accuracy, while improving the backdoor model accuracy scores.

In [ ]:
defenses = ["gaussian_augmentation", "spatial_smoothing", "jpeg_compression"]
defense = defenses[1]

response_poison_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point=defense,
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={DATASET_DIR}/testing",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            f"-P dataset_run_id={response_gen_poison_le_net_test['mlflowRunId']}",
            "-P dataset_tar_name=adversarial_poison.tar.gz",
            "-P dataset_name=adv_poison_data",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_gen_poison_le_net_test["jobId"],
)


print(f"{defense} defense (LeNet architecture) job submitted")
print("")
pprint.pprint(response_poison_def)
print("")

response_poison_def = get_run_id(response_poison_def)

In [ ]:
# Inference: Poisoned model on poisoned test images.
response_infer_pos_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_poison_def['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_data_poison_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            f"-P adv_tar_name={defense}_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_poison_def["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_pos_model)
print_mlflow_results(response_infer_pos_model)

In [ ]:
# Inference: Regular model on poisoned test images.
response_infer_reg_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_poison_def['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            f"-P adv_tar_name={defense}_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_poison_def["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_reg_model)
print_mlflow_results(response_infer_reg_model)


<a id='querying_cell'></a>
## Querying the MLFlow Tracking Service

Currently the lab API can only be used to register experiments and start jobs, so if users wish to extract their results programmatically, they can use the `MlflowClient()` class from the `mlflow` Python package to connect and query their results.
Since we captured the run ids generated by MLFlow, we can easily retrieve the data logged about one of our jobs and inspect the results.
To start the client, we simply need to run,

In [ ]:
mlflow_client = MlflowClient()

The client uses the environment variable `MLFLOW_TRACKING_URI` to figure out how to connect to the MLFlow Tracking Service, which we configured near the top of this notebook.
To query the results of one of our runs, we just need to pass the run id to the client's `get_run()` method.
As an example, let's query the run results for the patch attack applied to the LeNet-5 architecture,

In [ ]:
run_le_net = mlflow_client.get_run(response_le_net_train["mlflowRunId"])

If the request completed successfully, we should now be able to query data collected during the run.
For example, to review the collected metrics, we just use,

In [ ]:
pprint.pprint(run_le_net.data.metrics)

To review the run's parameters, we use,

In [ ]:
pprint.pprint(run_le_net.data.params)

To review the run's tags, we use,

In [ ]:
pprint.pprint(run_le_net.data.tags)

There are many things you can query using the MLFlow client.
[The MLFlow documentation gives a full overview of the methods that are available](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient).